Predict 5-Year Career Longevity for NBA Rookies

y = 0 if career years played < 5 and y = 1 if career years played >= 5

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


#Deep Learning
import torch 
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset 

In [ ]:
# Loading CSV
nba = pd.read_csv('/content/nba_logreg.csv')
nba.head()

,Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0
2,JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0
3,Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0
4,Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0


In [ ]:
nba.rename(columns={'FG%':'FG', '3P Made':'3P_Made', '3P%':'3P', 'FT%':'FT'}, inplace=True)
nba.head()

,Name,GP,MIN,PTS,FGM,FGA,FG,3P_Made,3PA,3P,FTM,FTA,FT,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0
2,JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0
3,Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0
4,Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0


In [ ]:
# Separating Features from NBA
features = nba.drop('TARGET_5Yrs', axis=1)
features.head()

,Name,GP,MIN,PTS,FGM,FGA,FG,3P_Made,3PA,3P,FTM,FTA,FT,OREB,DREB,REB,AST,STL,BLK,TOV
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6
2,JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0
3,Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0
4,Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8


In [ ]:
nba.isnull().sum()

Name            0
GP              0
MIN             0
PTS             0
FGM             0
FGA             0
FG              0
3P_Made         0
3PA             0
3P             11
FTM             0
FTA             0
FT              0
OREB            0
DREB            0
REB             0
AST             0
STL             0
BLK             0
TOV             0
TARGET_5Yrs     0
dtype: int64

In [ ]:
nba['3P'].fillna((nba['3P'].mean()), inplace=True)

In [ ]:
nba.isnull().sum()

Name           0
GP             0
MIN            0
PTS            0
FGM            0
FGA            0
FG             0
3P_Made        0
3PA            0
3P             0
FTM            0
FTA            0
FT             0
OREB           0
DREB           0
REB            0
AST            0
STL            0
BLK            0
TOV            0
TARGET_5Yrs    0
dtype: int64

In [ ]:
# Separating Target from NBA
target = nba['TARGET_5Yrs']
target.head()

0    0.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: TARGET_5Yrs, dtype: float64

In [ ]:
features.dtypes

Name        object
GP           int64
MIN        float64
PTS        float64
FGM        float64
FGA        float64
FG         float64
3P_Made    float64
3PA        float64
3P         float64
FTM        float64
FTA        float64
FT         float64
OREB       float64
DREB       float64
REB        float64
AST        float64
STL        float64
BLK        float64
TOV        float64
dtype: object

In [ ]:
features.drop('Name', inplace=True, axis=1 )

In [ ]:
features.dtypes

GP           int64
MIN        float64
PTS        float64
FGM        float64
FGA        float64
FG         float64
3P_Made    float64
3PA        float64
3P         float64
FTM        float64
FTA        float64
FT         float64
OREB       float64
DREB       float64
REB        float64
AST        float64
STL        float64
BLK        float64
TOV        float64
dtype: object

In [ ]:
# Normalizing features
scalar = StandardScaler()
features = scalar.fit_transform(features) #creates a Numpy array
features.shape

(1340, 19)

In [ ]:
#  Creating a class of DataSet
class TabDataSet(Dataset):
  def __init__(self, features, target):
    self.x = features
    self.y = target
    self.x = torch.from_numpy(self.x).type(torch.float32)
    self.y = torch.from_numpy(self.y.values)

  def __len__(self): return len(self.x)
  
  def __getitem__(self, idx): return self.x[idx], self.y[idx]

In [ ]:
# Function to get a DataLoader
def get_dataloader(features, target, bs):
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

  #DataSet
  train_ds = TabDataSet(X_train, y_train)
  test_ds = TabDataSet(X_test, y_test)

  #DataLoader
  train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
  test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)

  return train_dl, test_dl

In [ ]:
bs=58
train_dl, test_dl = get_dataloader(features, target, bs)

In [ ]:
#  Finding first element of train_dl
def first(x):
  try: return next(iter(x))
  except StopIteration: return None

In [ ]:
xb, yb = first(train_dl)
xb.shape, yb.shape

(torch.Size([58, 19]), torch.Size([58]))

In [ ]:
def get_model(n_in, n_out):
  return nn.Sequential(
      nn.Linear(n_in, 9),
      nn.ReLU(),
      nn.Dropout(p=0.2),
      nn.Linear(9, n_out)
  )

In [ ]:
model = get_model(19,1)
model

Sequential(
  (0): Linear(in_features=19, out_features=9, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=9, out_features=1, bias=True)
)

In [ ]:
def loss_func(prediction, target):
  prediction = prediction.sigmoid()
  return torch.where(target==1, 1-prediction, prediction).mean()


In [ ]:
def batch_accuracy(xb, yb):
  preds  = torch.sigmoid(xb)
  correct = (preds>0.5) == yb
  return correct.float().mean()

In [ ]:
def validate_epoch(model, epoch, metric):
  accs = [batch_accuracy(model(xb), yb) for xb,yb in train_dl]
  return round(torch.stack(accs).mean().item(), 4)

In [ ]:
def train_model(model, epoch, metric):
  for i in range(epoch):
    for  xb, yb in train_dl:
      preds =  model(xb.float())
      loss = loss_func(preds, yb)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    print(validate_epoch(model, epoch, metric), end = ' ')
  print('\n')   


In [ ]:
from torch import optim

In [ ]:
for i in [0.1, 0.01, 0.001, 0.0001, 0.00001]:
  optimizer  = optim.Adam(model.parameters(), lr=i)
  train_model(model, 5, metric=batch_accuracy)

0.3876 0.3808 0.3789 0.3808 0.3837 

0.3847 0.3818 0.3798 0.3798 0.3827 

0.3827 0.3847 0.3808 0.3789 0.3866 

0.3808 0.3827 0.3886 0.3837 0.3818 

0.3837 0.3847 0.3818 0.3808 0.3847 



In [ ]:
test_data  = features[4]
out_predict = model(torch.Tensor(test_data).float())
out_predict.argmax()

tensor(0)